# Make animation of locations of keypoints in poses of a person.

## Import packages.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import matplotlib.animation as animation
from matplotlib.ticker import MultipleLocator
from IPython.display import HTML

## Show text of the first json file from the video.

- Each json file refers to multiple persons. But, this analysis is only for the first person.

In [2]:
with open('J:/Hack-2502-Devpost-QualcommSnapdragon/keypoints_Qualcomm_CRD8380X/video_000000000000_keypoints.json', 'r') as zfil:
    zful = zfil.read().replace('\n', '').replace(" ","")
zful 

'{"version":1.3,"people":[{"person_id":[-1],"pose_keypoints_2d":[263.022,100.421,0.201275,270.889,125.844,0.904525,249.268,129.748,0.850839,243.432,165.007,0.752632,243.456,194.484,0.596226,290.522,121.943,0.896464,306.134,157.207,0.891701,312,188.602,0.856509,280.641,194.392,0.832359,266.932,194.511,0.813259,278.743,241.465,0.841595,302.164,265.015,0.82922,292.48,190.581,0.820523,300.255,239.524,0.816123,304.11,280.634,0.819429,259.154,98.4266,0.182094,266.929,94.556,0.191094,257.079,100.367,0.12363,276.717,96.4419,0.242894,296.328,290.478,0.721389,304.165,288.54,0.768734,306.065,282.639,0.617369,294.37,280.632,0.532733,290.47,278.669,0.551125,306.14,266.9,0.574465],"face_keypoints_2d":[],"hand_left_keypoints_2d":[],"hand_right_keypoints_2d":[],"pose_keypoints_3d":[],"face_keypoints_3d":[],"hand_left_keypoints_3d":[],"hand_right_keypoints_3d":[]},{"person_id":[-1],"pose_keypoints_2d":[0,0,0,1060.34,65.1192,0.905688,1072.12,61.1983,0.874789,1083.91,84.6741,0.708469,1078.06,102.372,0.44

## Extract coordinates of 15 keypoints of the first person.

- Keypoints extracted: Nose, Neck, RShoulder, RElbow, RWrist, LShoulder, LElbow, LWrist, MidHip, RHip, RKnee, RAnkle, LHip, LKnee, LAnkle

- Keypoints not extracted: REye, LEye, REar, LEar, LBigToe, LSmallToe, LHeel, RBigToe, RSmallToe, RHeel, Background

In [3]:
zbeg = zful.find('2d":[') + 6
zend = zful.find('],"face')
zstr = zful[zbeg:zend]
zstr

'63.022,100.421,0.201275,270.889,125.844,0.904525,249.268,129.748,0.850839,243.432,165.007,0.752632,243.456,194.484,0.596226,290.522,121.943,0.896464,306.134,157.207,0.891701,312,188.602,0.856509,280.641,194.392,0.832359,266.932,194.511,0.813259,278.743,241.465,0.841595,302.164,265.015,0.82922,292.48,190.581,0.820523,300.255,239.524,0.816123,304.11,280.634,0.819429,259.154,98.4266,0.182094,266.929,94.556,0.191094,257.079,100.367,0.12363,276.717,96.4419,0.242894,296.328,290.478,0.721389,304.165,288.54,0.768734,306.065,282.639,0.617369,294.37,280.632,0.532733,290.47,278.669,0.551125,306.14,266.9,0.574465'

## Arrange keypoints for creating a wireframe through Matplotlib.

In [4]:
zwir = [1,2,3,4,5,4,3,2,6,7,8,7,6,2,9,10,11,12,11,10,9,13,14,15]

## Define a function for creating two arrays of x and y coordinates.

In [5]:
def fpts(p):
    with open('J:/Hack-2502-Devpost-QualcommSnapdragon/keypoints_Qualcomm_CRD8380X/video_000000000' + str(9999000+p).replace('9999','') + '_keypoints.json', 'r') as zfil:
        zful = zfil.read().replace('\n', '').replace(" ","")
    zbeg = zful.find('2d":[') + 5
    zend = zful.find('],"face')
    zstr = zful[zbeg:zend]
    zarr = zstr.split(',')
    zxps = [int(float(zarr[3*m-3])) for m in zwir]
    zyps = [int(float(zarr[3*m-2])) for m in zwir]
    return zxps, zyps

## Create two arrays of x and y coordinates from all json files.

- Ignore records not belonging to the first peson, by using criteria of locations of the keypoints.

In [6]:
zxar = []
zyar = []

zxp0, zyp0 = fpts(0)

for i in range(205):
    zxps, zyps = fpts(i)
    if (zxps[0]>0) and (zyps[0]>0) and (zyps[1]>zyps[0]) and (zyps[4]>zyps[3]) and (zyps[10]>zyps[9]) and (zyps[16]>zyps[15]) and (zyps[17]>zyps[16]) and (zyps[22]>zyps[21]) and (zyps[23]>zyps[22]) and (zyps[1]<zyps[0]+50) and (zyps[4]<zyps[3]+50) and (zyps[10]<zyps[9]+50) and (zyps[16]<zyps[15]+50) and (zyps[17]<zyps[16]+50) and (zyps[22]<zyps[21]+50) and (zyps[23]<zyps[22]+50):
        zxar = zxar + [[zxps[j]-zxps[0] for j in range(24)]]
        zyar = zyar + [[-2*zyps[j]+2*zyps[0] for j in range(24)]]
        zxp0 = zxps[0]
        zyp0 = zyps[0]

len(zxar)

44

## Create an animation file and html using Matplotlib.

In [7]:
%%capture
fig, ax = plt.subplots(figsize=(4,8))
plt.xticks([-100,150,150,-100])
plt.yticks([50,50,-400,-400])
x = []
y = []
plt.plot(x, y, "green", linestyle='-')
plt.text(0,0,"x", font="Times New Roman")

z, = ax.plot(x, y, 'go')

def update(p):
    z.set_data(zxar[p], zyar[p])
    z.set_linestyle('-')
    return z

anim = animation.FuncAnimation(fig, update, frames=range(len(zxar))) # , fargs=(z), interval=1000)
anim.save('x.gif', writer='Pillow', fps=15)
zhtm = HTML(anim.to_jshtml())

plt.show()

## Display the animation through html.

In [8]:
zhtm